In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.ticker import MultipleLocator
import panel as pn
import hvplot.pandas
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

In [2]:
pn.extension(design='material', theme='default')

In [3]:
df = pd.read_csv('jobs_cleaned_1.csv')

In [4]:
qualifications = pd.read_csv('qualifications.csv')

In [5]:
del(qualifications['Unnamed: 0'])
qualifications = qualifications.rename(columns={'0': 'Qualifications'})

In [6]:
Type = pd.read_csv('type.csv')

In [7]:
del(Type['Unnamed: 0'])
Type = Type.rename(columns={'0': 'Type'})

In [8]:
benefits = pd.read_csv('benefits.csv')

In [9]:
del(benefits['Unnamed: 0'])
benefits = benefits.rename(columns={'0': 'Benefits'})

In [10]:
range_slider_scatter = pn.widgets.IntRangeSlider(name='Index Range', start=0, end=935, value=(0,935))
column_selector_scatter = pn.widgets.Select(name='Column', options=['num_Qualifications', 'num_Benefits', 'Rating', 'Salary_Begin', 'Salary_End', 'Date_Posted_Complete', 'Posted_Month', 'Type', 'Job_simp', 'Salary_Average'], value='num_Qualifications')
color_scatter = pn.widgets.ColorPicker(value='navy')

def create_scatter(column, ranges_scatter, color):
    data = df[column][ranges_scatter[0]:ranges_scatter[1]]
    return data.reset_index().hvplot.scatter(x='index', y=column, color=color, size=20, alpha=.5)

In [11]:
column_selector_hist = pn.widgets.Select(name='column', options=['Rating', 'num_Benefits', 'num_Qualifications', 'Salary_Begin', 'Salary_Average', 'Salary_End'], value='Rating')
color_hist = pn.widgets.ColorPicker(value='#FB93DA')

def create_hist(column, color):
    if column == 'num_Benefits' or column == 'num_Qualifications':
        data = df[column].apply(lambda x: round(x))
    else:
        data = df[column]
    return data.reset_index().hvplot.hist(column, color=color)

In [12]:
import hvplot.xarray  # noqa
import xarray as xr

column_selector_bar = pn.widgets.Select(name='Column', options=['Job_simp', 'Company', 'Date_Posted_Complete', 'Type', 'Qualifications', 'Benefits'], value='Company')

range_slider_bar = pn.widgets.IntRangeSlider(name='Index Range', start=0, end=50, value=(0,21))

def set_range(event):
    if column_selector_bar.value=='Type':
        range_slider_bar.end=len(Type[column_selector_bar.value].value_counts())
    elif column_selector_bar.value=='Qualifications':
        range_slider_bar.end=len(qualifications[column_selector_bar.value].value_counts())
    elif column_selector_bar.value=='Benefits':
        range_slider_bar.end=len(benefits[column_selector_bar.value].value_counts())
    else:
        range_slider_bar.end=len(df[column_selector_bar.value].value_counts())

color_bar = pn.widgets.ColorPicker(value='blue')

column_selector_bar.param.watch(set_range, 'value')

def create_bar(column, color, ranges):
    if column=='Type':
        data=Type[column].value_counts()[ranges[0]:ranges[1]]
    elif column=='Qualifications':
        data=qualifications[column].value_counts()[ranges[0]:ranges[1]]
    elif column=='Benefits':
        data=benefits[column].value_counts()[ranges[0]:ranges[1]]
    else:
        data=df[column].value_counts()[ranges[0]:ranges[1]]
    
    return data.reset_index().hvplot.bar(x=column, rot=90, height=700, width=580, color=color)

In [13]:
rate_input = pn.widgets.FloatInput(name='Rating', start=0, end=5, step=1e-1)

def create_dataframe(rating):
    return df[df['Rating']==rating][['Job', 'Salary_Begin', 'Salary_End', 'Company', 'Rating', 'Type', 'Date_Posted_Complete']]

In [14]:
pn.Column(
    pn.Row(
        pn.GridBox(
            pn.Column(
                pn.Row(pn.Spacer(width=60), column_selector_scatter, color_scatter),
                pn.Row(pn.Spacer(width=60), range_slider_scatter),
                pn.Row(pn.panel(pn.bind(create_scatter, column_selector_scatter, range_slider_scatter, color_scatter), height=500, width=600))
            )
        ),
        pn.GridBox(
            pn.Column(
                pn.Row(pn.Spacer(width=100), column_selector_hist, color_hist),
                pn.Spacer(height=53),
                pn.Row(pn.Spacer(width=65), pn.panel(pn.bind(create_hist, column_selector_hist, color_hist), height=500, width=600))
            )
        )
    ),
    pn.Spacer(height=100),
    pn.Row(
        pn.GridBox(
            pn.Column(
                pn.Row(pn.Spacer(width=60), column_selector_bar, color_bar),
                pn.Row(pn.Spacer(width=60), range_slider_bar),
                pn.Row(pn.panel(pn.bind(create_bar, column_selector_bar, color_bar, range_slider_bar), height=500, width=600))
            )
        ),
        pn.GridBox(
            pn.Column(
                pn.Row(pn.Spacer(width=100), rate_input),
                pn.Spacer(height=53),
                pn.Row(pn.Spacer(width=100),pn.panel(pn.bind(create_dataframe, rate_input), height=500, width=600))
            )
        )
    )
).servable()

Column(design=<class 'panel.theme.materi...)
    [0] Row(design=<class 'panel.theme.materi...)
        [0] GridBox(design=<class 'panel.theme.materi...)
            [0] Column(design=<class 'panel.theme.materi...)
                [0] Row(design=<class 'panel.theme.materi...)
                    [0] Spacer(design=<class 'panel.theme.materi..., width=60)
                    [1] Select(design=<class 'panel.theme.materi..., name='Column', options=['num_Qualifications', ...], value='num_Qualifications')
                    [2] ColorPicker(design=<class 'panel.theme.materi..., value='navy')
                [1] Row(design=<class 'panel.theme.materi...)
                    [0] Spacer(design=<class 'panel.theme.materi..., width=60)
                    [1] IntRangeSlider(design=<class 'panel.theme.materi..., end=935, name='Index Range', value=(0, 935), value_end=935)
                [2] Row(design=<class 'panel.theme.materi...)
                    [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi..., height=500, sizing_mode='fixed', width=600)
        [1] GridBox(design=<class 'panel.theme.materi...)
            [0] Column(design=<class 'panel.theme.materi...)
                [0] Row(design=<class 'panel.theme.materi...)
                    [0] Spacer(design=<class 'panel.theme.materi..., width=100)
                    [1] Select(design=<class 'panel.theme.materi..., name='column', options=['Rating', 'num_Benefits',...], value='Rating')
                    [2] ColorPicker(design=<class 'panel.theme.materi..., value='#FB93DA')
                [1] Spacer(design=<class 'panel.theme.materi..., height=53)
                [2] Row(design=<class 'panel.theme.materi...)
                    [0] Spacer(design=<class 'panel.theme.materi..., width=65)
                    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi..., height=500, sizing_mode='fixed', width=600)
    [1] Spacer(design=<class 'panel.theme.materi..., height=100)
    [2] Row(design=<class 'panel.theme.materi...)
        [0] GridBox(design=<class 'panel.theme.materi...)
            [0] Column(design=<class 'panel.theme.materi...)
                [0] Row(design=<class 'panel.theme.materi...)
                    [0] Spacer(design=<class 'panel.theme.materi..., width=60)
                    [1] Select(design=<class 'panel.theme.materi..., name='Column', options=['Job_simp', 'Company', ...], value='Company')
                    [2] ColorPicker(design=<class 'panel.theme.materi..., value='blue')
                [1] Row(design=<class 'panel.theme.materi...)
                    [0] Spacer(design=<class 'panel.theme.materi..., width=60)
                    [1] IntRangeSlider(design=<class 'panel.theme.materi..., end=50, name='Index Range', value=(0, 21), value_end=21)
                [2] Row(design=<class 'panel.theme.materi...)
                    [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi..., height=500, sizing_mode='fixed', width=600)
        [1] GridBox(design=<class 'panel.theme.materi...)
            [0] Column(design=<class 'panel.theme.materi...)
                [0] Row(design=<class 'panel.theme.materi...)
                    [0] Spacer(design=<class 'panel.theme.materi..., width=100)
                    [1] FloatInput(design=<class 'panel.theme.materi..., end=5, name='Rating', start=0)
                [1] Spacer(design=<class 'panel.theme.materi..., height=53)
                [2] Row(design=<class 'panel.theme.materi...)
                    [0] Spacer(design=<class 'panel.theme.materi..., width=100)
                    [1] ParamFunction(function, _pane=DataFrame, defer_load=False, design=<class 'panel.theme.materi..., height=500, sizing_mode='fixed', width=600)

In [15]:
'''
pn.Column(
    pn.Row(pn.Spacer(width=100), column_selector_scatter, color_scatter, pn.Spacer(width=280), column_selector_hist, color_hist),
    pn.Row(pn.Spacer(width=100), range_slider_scatter),
    pn.Row(pn.panel(pn.bind(create_scatter, column_selector_scatter, range_slider_scatter, color_scatter), height=500, width=600), pn.Spacer(width=140), pn.panel(pn.bind(create_hist, column_selector_hist, color_hist), height=500, width=600)),
    pn.Spacer(height=100),
    pn.Row(pn.Spacer(width=100), column_selector_bar, color_bar, pn.Spacer(width=280), rate_input),
    pn.Row(pn.Spacer(width=100), range_slider_bar),
    pn.Row(pn.Spacer(width=37), pn.panel(pn.bind(create_bar, column_selector_bar, color_bar, range_slider_bar), height=500, width=600), pn.Spacer(width=140), pn.panel(pn.bind(display_dataframe, rate_input), height=500, width=600))
    
).servable()
'''

'\npn.Column(\n    pn.Row(pn.Spacer(width=100), column_selector_scatter, color_scatter, pn.Spacer(width=280), column_selector_hist, color_hist),\n    pn.Row(pn.Spacer(width=100), range_slider_scatter),\n    pn.Row(pn.panel(pn.bind(create_scatter, column_selector_scatter, range_slider_scatter, color_scatter), height=500, width=600), pn.Spacer(width=140), pn.panel(pn.bind(create_hist, column_selector_hist, color_hist), height=500, width=600)),\n    pn.Spacer(height=100),\n    pn.Row(pn.Spacer(width=100), column_selector_bar, color_bar, pn.Spacer(width=280), rate_input),\n    pn.Row(pn.Spacer(width=100), range_slider_bar),\n    pn.Row(pn.Spacer(width=37), pn.panel(pn.bind(create_bar, column_selector_bar, color_bar, range_slider_bar), height=500, width=600), pn.Spacer(width=140), pn.panel(pn.bind(display_dataframe, rate_input), height=500, width=600))\n    \n).servable()\n'